In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import numpy as np


ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# download pretrained model resnet18
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)

# replace the last layer of resnet18 with a Sequential layer (lin, relu, drop, lin)
resnet18.fc = nn.Sequential(
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)




In [ ]:
# define a loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# train the network
def get_accuracy(model, val_dl, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_dl:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


def train(model, loss_fn, optimizer, train_dl, val_dl, epochs=10, device="gpu"):
    for epoch in range(epochs):
        # Training
        model.train()
        for xb, yb in train_dl:
            # move data to device
            xb = xb.to(device)
            yb = yb.to(device)

            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)

            # Perform gradient descent
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Evaluation
        model.eval()
        with torch.no_grad():
            # Get validation loss
            for xb, yb in val_dl:
                # move data to device
                xb = xb.to(device)
                yb = yb.to(device)
                pred = model(xb)
                loss = loss_fn(pred, yb)
                val_loss = loss.item()
            print("Epoch:", epoch, "Val Loss:", val_loss)
            # print accuracy
            print("Accuracy:", get_accuracy(model, val_dl, device))

In [ ]:

import os

batch_size = 32
img_dims=(224, 224)
img_transforms = transforms.Compose([
    transforms.Resize(img_dims),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

img_test_transforms = transforms.Compose([
    transforms.Resize(img_dims),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_path = "../input/microsoft-catsvsdogs-dataset/PetImages"

# data folder: Cat and Dog
# load data and split into train (90%) and validation (10%)
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from PIL import Image

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        print(path)
        return False

dataset = ImageFolder(data_path, transform=img_transforms, is_valid_file=check_image)
train_ds, val_ds = random_split(dataset, [len(dataset)//10*9, len(dataset)//10*1])

# create data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)





In [ ]:
train(resnet18, loss_fn, optimizer, train_dl, val_dl, epochs=10, device="cpu")

In [ ]:
# get accuracy
get_accuracy(resnet18, val_dl, "cpu")

In [ ]:
#save model
torch.save(resnet18.state_dict(), "resnet18.pth")